# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [4]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
import pandas as pd
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM albums LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [5]:
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`


In [26]:

result = %sql SELECT Name, Title as Album FROM tracks t LEFT JOIN albums a ON t.AlbumID = a.AlbumId LIMIT 5;
 
display(result)


 * sqlite:///chinook.sqlite
Done.


Name,Album
For Those About To Rock (We Salute You),For Those About To Rock We Salute You
Balls to the Wall,Balls to the Wall
Fast As a Shark,Restless and Wild
Restless and Wild,Restless and Wild
Princess of the Dawn,Restless and Wild


2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`


In [29]:

result = %sql SELECT COUNT(CustomerID) AS TotalCustomer, Country  FROM customers GROUP BY Country ORDER BY TotalCustomer DESC;
 
display(result)


 * sqlite:///chinook.sqlite
Done.


TotalCustomer,Country
13,USA
8,Canada
5,France
5,Brazil
4,Germany
3,United Kingdom
2,Portugal
2,India
2,Czech Republic
1,Sweden


3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`


In [30]:

result = %sql SELECT COUNT(TrackID) AS TrackCount, g.Name FROM tracks t LEFT JOIN genres g ON t.GenreId = g.GenreId GROUP BY g.GenreId ORDER BY TrackCount DESC LIMIT 5;
 
display(result)


 * sqlite:///chinook.sqlite
Done.


TrackCount,Name
1297,Rock
579,Latin
374,Metal
332,Alternative & Punk
130,Jazz


4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`


In [50]:
query = """
        SELECT AVG(InvoiceTotal ) 
        FROM (SELECT SUM(Quantity*UnitPrice) AS InvoiceTotal 
        FROM invoice_items i
        LEFT JOIN invoices inv ON i.InvoiceId = inv.InvoiceId
        GROUP BY i.invoiceId, inv.CustomerId) total
 

        """
result = %sql $query
display(result)


 * sqlite:///chinook.sqlite
Done.


AVG(InvoiceTotal )
5.651941747572825


5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [57]:
query = """
        SELECT SUM(Quantity*UnitPrice) AS Total, c.FirstName, c.LastName
        FROM invoice_items i
        LEFT JOIN invoices inv ON i.InvoiceId = inv.InvoiceId
        LEFT JOIN customers c ON inv.CustomerId = c.CustomerId
        GROUP BY inv.CustomerId
        ORDER BY total DESC LIMIT 1
        """
result = %sql $query
display(result)


 * sqlite:///chinook.sqlite
Done.


Total,FirstName,LastName
49.620000000000005,Helena,Holý
